In [ ]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq groq==0.13.0 --progress-bar off
!pip install -qqq python-dotenv==1.0.1 --progress-bar off
!pip install groq
!pip install nltk

In [ ]:
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize

nltk.download('punkt')

In [ ]:
def chunk_transcript(data: pd.DataFrame) -> pd.DataFrame:
    
    # Ensure 'Transkript' column has no NaN values
    data['Transkript'] = data['Transkript'].fillna("").astype(str)

    # Chunking logic
    chunked_data = []
    current_speaker = None
    current_text = ""
    initial_timestamp = ""

    for index, row in data.iterrows():
        speaker = row['Sprecher']
        transcript = row['Transkript']
        timestamp = row['Timecode'] 

        if speaker != current_speaker:
            # Save previous chunk if exists
            if current_speaker is not None:
                chunked_data.append({
                    'Speaker': current_speaker,
                    'Transcript': current_text.strip(),
                    'Initial_Timestamp' : initial_timestamp,
                    'Current_Timestamp' : timestamp
                })
            
            # Start a new chunk
            current_speaker = speaker
            current_text = transcript
            initial_timestamp = timestamp 
        else:
            # Continue appending to the same speaker's chunk
            current_text += " " + transcript if isinstance(transcript, str) else ""

    # Save the last chunk
    if current_speaker is not None:
        chunked_data.append({
            'Speaker': current_speaker,
            'Transcript': current_text.strip(),
            'Initial_Timestamp' : initial_timestamp,
            'Current_Timestamp' : timestamp
        })

    # Convert to DataFrame
    chunked_df = pd.DataFrame(chunked_data)
    
    return chunked_df

In [ ]:
def chunk_by_sentence(chunked_df: pd.DataFrame, min_tokens=256, max_tokens=512) -> pd.DataFrame:
    """
    Further splits chunks by sentence while ensuring each chunk is within a token range.

    Args:
        chunked_df (pd.DataFrame): Input DataFrame with 'Speaker' and 'Transcript' columns.
        min_tokens (int): Minimum number of tokens per chunk.
        max_tokens (int): Maximum number of tokens per chunk.

    Returns:
        pd.DataFrame: DataFrame with sentence-based chunked transcripts.
    """

    # Function to count tokens (approximate, assuming 1 word ≈ 1.2 tokens)
    def count_tokens(text):
        return len(text.split()) * 1.2  # Rough estimate

    # Initialize list for final merged chunks
    merged_chunks = []
    temp_chunk = []
    temp_token_count = 0
    speaker = None
    initial_timestamp = ""
    final_timestamp = ""

    # Process each row
    for _, row in chunked_df.iterrows():
        sentence = row['Transcript']
        sentence_tokens = count_tokens(sentence)

        # If adding this chunk keeps us within MAX_TOKENS
        if temp_token_count + sentence_tokens <= max_tokens:
            if not temp_chunk:
                speaker = row['Speaker']  # Store speaker only for new chunks
                initial_timestamp = row['Initial_Timestamp']
            temp_chunk.append(sentence)
            temp_token_count += sentence_tokens
        else:
            # Save the previous chunk before starting a new one
            if temp_chunk:
                merged_chunks.append({
                    'Speaker': speaker,
                    'Transcript': " ".join(temp_chunk),
                    'Initial_Timestamp' : initial_timestamp,
                    'Current_Timestamp' : row['Current_Timestamp']
                })

            # Start a new chunk with the current sentence
            temp_chunk = [sentence]
            temp_token_count = sentence_tokens
            speaker = row['Speaker']
            initial_timestamp = row['Initial_Timestamp']
            final_timestamp = row['Current_Timestamp']

    # Save last chunk if any content remains
    if temp_chunk:
        merged_chunks.append({
            'Speaker': speaker,
            'Transcript': " ".join(temp_chunk),
            'Initial_Timestamp' : initial_timestamp,
            'Current_Timestamp' : final_timestamp
        })

    # Convert to DataFrame
    final_merged_df = pd.DataFrame(merged_chunks)
    
    return final_merged_df

In [ ]:
from groq import Groq
import pandas as pd 

api_key = {your api_key}

# Initialize the Groq client
client = Groq(api_key=api_key)

print("Groq client initialized successfully!")

In [ ]:
def extract_metadata(client, model_name: str, chunks: pd.DataFrame) -> pd.DataFrame:
    """
    Extract metadata from multiple chunks of a German transcript using the specified model.
    
    Args:
        client: The Groq client object.
        model_name: The name of the model to use (e.g., 'llama3-8b-8192').
        chunks: A DataFrame with 'Speaker' and 'Transcript' columns.
    
    Returns:
        A DataFrame containing the extracted metadata for all chunks.
    """
    all_metadata = []
    num_chunks = len(chunks)

    for i, row in chunks.iterrows():  # Iterate over DataFrame rows
        speaker = row["Speaker"]
        transcript = row["Transcript"]
        timestamp = row['Timestamp']

        print(f"Processing chunk {i+1}/{len(chunks)} for Speaker: {speaker}...")

        # Eingabeaufforderung zur Metadatenextraktion
        prompt = f"""

        Extrahieren Sie die folgenden Informationen aus dem Transkript und geben Sie die einsilbige Antwort auf Deutsch ein.  
        Wenn die Antwort nicht gefunden wurde, geben Sie stattdessen %%% zurück.  
        
        Wenn es mehrere Antworten gibt, wählen Sie eine einzelne geeignete Antwort entsprechend dem Spaltennamen und der unten angegebenen Beschreibung.  
        Fügen Sie nach jedem Wert in Klammern den entsprechenden {timestamp} hinzu.  
        Wenn die generierten Antworten auf kontextuellem Verständnis basieren und nicht im {transcript} enthalten sind, fügen Sie den {timestamp} nicht hinzu.  
        Fügen Sie außerdem den {timestamp} nicht hinzu, wenn die Antwort %%% ist oder nicht gefunden wurde. 
        
        
        
        
        NAME: Der vollständige Name der Person (Vor- und Nachname, falls verfügbar).
        JAHRGANG: Das Geburtsjahr der Person.
        ORT: Der Geburtsort der Person.
        GESCHLECHT: Das Geschlecht der Person (z.B. männlich, weiblich, divers).
        BERUF: Der aktuelle Beruf der Person.

        VAT_JG: Geburtsjahr des Vaters.
        VAT_KONFESSION: Religion des Vaters.
        VAT_HERKUN: Herkunft des Vaters.
        VAT_SCHULE: Schulbildung des Vaters.
        VAT_AUSBIL: Ausbildung des Vaters.
        VAT_STAND: Beruflicher Status des Vaters.
        VAT_POLOR: Politische Orientierung des Vaters.

        DATENBOGEN: Während des Interviews verwendetes Datenblatt oder Formular.
        KURZBESCHR: Kurzbeschreibung des Interviews.
        TITEL: Titel des Dokuments oder der Aufzeichnung.
        DAUER: Dauer des Interviews.
        INTERVIEWE: Name des Interviewers.
        TIPPER: Schreibkraft oder Person, die das Interview transkribiert hat.
        Segmentierung: Segmentierung (Kategorien oder Abschnitte innerhalb des Interviews).
        DATUM1, DATUM2, DATUM3: Wichtige Daten im Zusammenhang mit dem Interview.


        STRASSE: Straßenadresse.
        PLZ: Postleitzahl.
        TELEFON: Telefonnummer.
        
        GRUPPE: Gruppenzugehörigkeit (z. B. politische, soziale oder kulturelle Gruppe).
        BERUF: Beruf oder Tätigkeit.
        HEUT_FAMST: Aktueller Familienstand (z. B. ledig, verheiratet, verwitwet).

        FOTOS: Mit dem Datensatz verbundene Fotos.
        DOKUMENTE: Mit dem Datensatz verbundene Dokumente.
        VHS, DVD: Für die Aufzeichnung verwendete Medienformate (falls zutreffend).
        IBM-Server: Ob auf einem IBM-Server gespeichert.
        Cloud: Ob Daten in der Cloud gespeichert sind.
        ORIGCASSET, CASSKOPIEN: Originalkassette und Kopien.
        FESTPLATTE: Festplattenspeicherort.
        Dig Audiofiles: Mit dem Datensatz verbundene digitale Audiodateien.

        - Schulabsch: Schulbildungsniveau.
        - ABGEBROCHE: Abgebrochene Ausbildung.
        - WEITERBILD: Weiterbildung oder Ausbildung.
        - AUSBILDUNG: Berufsausbildung.
        - STAND: Aktueller beruflicher Status.
        - BERUFSWECH: Berufswechsel (falls zutreffend).
        - WANN_WECHS: Wann der Berufswechsel stattfand.
        - AUFABSTIEG: Karriereaufstieg oder -rückgang.
        - BERUFSBEGI: Berufseinstiegsjahr.
        - BERUFSENDE: Berufsabschlussjahr.
        - NICHTERWER: Nichterwerbstätiger Status (falls zutreffend).
        - GRÜNDE: Gründe für Nichtbeschäftigung.

        - FAM_STAND: Familienstand.
        - HEIRAT1JHR: Ehejahre (falls zutreffend).
          HEIRAT2JHR: Ehejahre (falls zutreffend).
          HEIRAT3JHR: Ehejahre (falls zutreffend).
        - SCHEID1JHR: Scheidungsjahre
          SCHEID2JHR: Scheidungsjahre.
        - VERWIT1JHR:Witwenjahre.
          VERWIT2JHR: Witwenjahre.
        - KINDERZAHL: Anzahl der Kinder.
        - GEB_JAHR1:Geburtsjahre der Kinder.
          GEB_JAHR2:Geburtsjahre der Kinder.
          GEB_JAHR_L: Geburtsjahre der Kinder.
        

        - POLOR_HEUT: Aktuelle politische Orientierung.
        - POL_KONVER: Politische Konversion (sofern vorhanden).
        - POL_ORIENT1:
          VON_BIS_1: Erste politische Orientierung und Dauer.
        - POL_ORIENT2:
          VON_BIS_2: Zweite politische Orientierung und Dauer.
        - GEW_VERBAN:
          VON_BIS_GV: Mitgliedschaft in Organisationen oder Gewerkschaften und Dauer.


        - JUGENDORG1:
          VON_BIS_J1: Erste Jugendorganisation und Dauer.
        - JUGENDORG2:
          VON_BIS_J2: Zweite Jugendorganisation und Dauer.
        - NS_ORGAN_1:
          VON_BISNS1: Erste NS-nahe Organisation und Dauer.
        - NS_ORGAN_2, VON_BISNS2: Zweite mit dem Nazismus verbundene Organisation und Dauer.
        - RAD_KLV_DV, VON_BISRAD: Anderes historisches Engagement und Dauer.


        - KRIEGSTEIL, VON_BIS_KR: Kriegsteilnahme und -dauer.
        - BES_BERICH: Kriegsbezogene Gebiete oder Erfahrungen.

        - MUTT_JG: Geburtsjahr der Mutter.
        - MUTT_KONFESSION: Religion der Mutter.
        - MUTT_HERKU: Herkunft der Mutter.
        - MUTT_SCHUL: Schulbildung der Mutter.
        - MUTT_AUSBI: Ausbildung der Mutter.
        - MUTT_STAND: Beruflicher Status der Mutter.
        - MUTT_POLOR: Politische Orientierung der Mutter.

        - VAT_JG, PART_JG: Geburtsjahr des Vaters/Partners.
        - VAT_KONFESSION, PART_KONFESSION: Religion.
        - VAT_HERKUN, PART_HERKU: Herkunft.
        - VAT_SCHULE, PART_SCHUL: Bildung.
        - VAT_AUSBIL, PART_AUSBI: Schulung.
        - VAT_STAND, PART_STAND: Professioneller Status.
        - VAT_POLOR, PART_POLOR: Politische Ausrichtung.
        - PART_BERUF: Beruf des Partners.
        
        - PART_PKONV: Politische Bekehrung des Partners.
        - PART_ENGAG: Soziales/politisches Engagement des Partners.
        - KRIT10: Anmerkungen oder Kritik.


        Transkript: 
        {transcript}
        
        Metadaten:
        STANDORT:
        ARCHIV ID:
        PROBANDNR:
        DOK_ART:
        ARCHIVORT:
        PROVENIENZ:
        SPERRUNG:
        ENTSTZEIT:
        ZEITUMFANG 1:
        NAME:
        VORNAME:
        ORT:
        FELD1:
        PSEUDONYM:
        GESCHLECHT:
        JAHRGANG:
        IPV:
        DATENBOGEN:
        KURZBESCHR:
        TITEL:
        STRASSE:
        PLZ:
        TELEFON:
        GRUPPE:
        BERUF:
        HEUT_FAMST:
        INTERVIEWE:
        TIPPER:
        SEGMENTIERUNG:
        DATUM1:
        DATUM2:
        DATUM3:
        DAUER:
        ONLINE:
        AUSDRUCKSART:
        UNKAUSDRUC:
        KORRAUSDRU:
        SCHLAGWORT:
        KURZBIOGRA:
        KURZPROTOK:
        FOTOS:
        DOKUMENTE:
        VHS:
        DVD:
        IBM SERVER:
        CLOUD:
        FORMAT CLOUD:
        DV:
        BETA:
        ORIGCASSET:
        CASSKOPIEN:
        FESTPLATTE:
        DIG AUDIOFILES:
        KONF_HEUTE:
        KONVERSION:
        WANN_KONV:
        HERKUNFT:
        WANN_ZUGEZ:
        GESCHWISTE:
        SCHULABSCH:
        ABGEBROCHE:
        WEITERBILD:
        AUSBILDUNG:
        STAND:
        WIRTSCHBER:
        BERUFSWECH:
        WANN_WECHS:
        AUFABSTIEG:
        BERUFSBEGI:
        BERUFSENDE:
        NICHTERWER:
        GRÜNDE:
        VON_BIS:
        ARBEITSLOS:
        VON_BIS_AL:
        FAM_STAND:
        HEIRAT1JHR:
        HEIRAT2JHR:
        HEIRAT3JHR:
        SCHEID1JHR:
        SCHEID2JHR:
        VERWIT1JHR:
        VERWIT2JHR:
        KINDERZAHL:
        GEB_JAHR1:
        GEB_JAHR2:
        GEB_JAHR_L:
        AUFABKIND:
        POLOR_HEUT:
        POL_KONVER:
        POLORIENT1:
        VON_BIS_1:
        POLORIENT2:
        VON_BIS_2:
        GEW_VERBAN:
        VON_BIS_GV:
        JUGENDORG1:
        VON_BIS_J1:
        JUGENDORG2:
        VON_BIS_J2:
        NS_ORGAN_1:
        VON_BISNS1:
        NS_ORGAN_2:
        VON_BISNS2:
        RAD_KLV_DV:
        VON_BISRAD:
        KRIEGSTEIL:
        VON_BIS_KR:
        BES_BERICH:
        MUTT_JG:
        MUTT_KONFESSION:
        MUTT_HERKU:
        MUTT_SCHUL:
        MUTT_AUSBI:
        MUTT_STAND:
        MUTT_POLOR:
        VAT_JG:
        VAT_KONFESSION:
        VAT_HERKUN:
        VAT_SCHULE:
        VAT_AUSBIL:
        VAT_STAND:
        VAT_POLOR:
        PART_JG:
        PART_KONFESSION:
        PART_HERKU:
        PART_SCHUL:
        PART_AUSBI:
        PART_STAND:
        PART_BERUF:
        PART_POLOR:
        PART_PKONV:
        PART_ENGAG:
        KRIT10:




        Fügen Sie keine Präambel ein.
        """



        # Generate response using the Groq client
        response = client.chat.completions.create(
            model=model_name,
            messages=[{"role": "user", "content": prompt}],
            temperature=0,
        )

        # Extract metadata from the response
        metadata = response.choices[0].message.content

        # Parse metadata into a dictionary
        extracted_metadata = {"Speaker": speaker}  # Store speaker info
        for line in metadata.split("\n"):
            if ":" in line:
                key, value = line.split(":", 1)
                extracted_metadata[key.strip()] = value.strip()

        # Remove "%%%%" values before storing
        extracted_metadata = {k: (v if v != "%%%" else "") for k, v in extracted_metadata.items()}

        if i < num_chunks - 1:  # If not the last iteration
            extracted_metadata = {k: v.strip() + "," if v else v.strip() for k, v in extracted_metadata.items()}

        # Append metadata for this chunk
        all_metadata.append(extracted_metadata)

    # Convert metadata list into a DataFrame
    return pd.DataFrame(all_metadata)


        # if i < num_chunks - 1:  # If not the last iteration
        #     for k, v in extracted_metadata.items():
        #         if v:  # If the value is not empty
        #             existing_values = set([extracted_metadata[k]][0].strip().split(" "))  # Convert existing values to a set
        #             # existing_values.add(v)  # Add new value (set avoids duplicates)
        #             extracted_metadata[k] = " | ".join(existing_values)  # Join back without duplicates



        # # Append metadata for this chunk
        # all_metadata.append(extracted_metadata)


    #     # Check for duplicates column-wise
    #     is_duplicate = False
    #     for prev_metadata in all_metadata:
    #         for key in extracted_metadata.keys():
    #             if key in prev_metadata and extracted_metadata[key] == prev_metadata[key]:  
    #                 is_duplicate = True
    #                 continue
    #         if is_duplicate:
    #             break  

    #     # If the entire metadata row is not a duplicate, append it
    #     if not is_duplicate:
    #         all_metadata.append(extracted_metadata)

    # Convert metadata list into a DataFrame
    # return pd.DataFrame(all_metadata)

        # Remove "%%%%" values before storing
        # extracted_metadata = {k: (v if v != "%%%" else "") for k, v in extracted_metadata.items()}

        # Check for duplicates column-wise
    #     cleaned_metadata = {"Speaker": speaker}  # Keep speaker info
    #     for key, value in extracted_metadata.items():
    #         if key == "Speaker":
    #             continue  # Skip speaker key check
    #         is_duplicate = any(value == prev_row.get(key, None) for prev_row in all_metadata)
            
    #         if not is_duplicate:  # Add only if value is unique
    #             cleaned_metadata[key] = value

    #         for key in cleaned_metadata.keys():
    #             values = [v for v in cleaned_metadata[key].split(" | ") if v]  # Remove empty values
    #             cleaned_metadata[key] = " | ".join(values) 


    #     # If there’s any new unique metadata, add it
    #     if len(cleaned_metadata) > 1:  
    #         all_metadata.append(cleaned_metadata)

    # # Convert metadata list into a DataFrame
    # return pd.DataFrame(all_metadata)

In [ ]:
import os
import time

folder_path = "Transcripts ADG0001-10"
MODEL = "llama-3.3-70b-versatile"

# List to store metadata for all files
all_metadata = []
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    if os.path.isfile(file_path) & filename.endswith(".csv"):
        print(f"\nProcessing file: {filename}")
        input_data = pd.read_csv(file_path, sep=None, engine='python')
        speaker_chunks_df = chunk_transcript(input_data)  # Stores speaker-based chunks
        final_chunks_df = chunk_by_sentence(speaker_chunks_df)
        final_chunks_df['Timestamp'] = final_chunks_df['Initial_Timestamp'] + " - " + final_chunks_df['Current_Timestamp'] 
        final_chunks_df.drop(columns=['Initial_Timestamp', "Current_Timestamp"], inplace=True)
        # print(final_chunks_df)
        

        # Extract metadata for the chunks
        llama_70b_responses = extract_metadata(client,MODEL , final_chunks_df)

        # Ensure that the response DataFrame contains metadata columns
        if not llama_70b_responses.empty:
            # Merge chunk outputs into a single row 
            merged_metadata = llama_70b_responses.apply(lambda col: ' '.join(col.dropna().astype(str)))
            
            for column in merged_metadata.index:
                unique_values = set([value.strip() for value in merged_metadata[column].strip().split(",")])
                list_unique_values = list(filter(None, unique_values))
                merged_metadata[column] = " | ".join(list_unique_values)

            # Add filename for reference
            # merged_metadata["Filename"] = filename  

            # Append to list
            all_metadata.append(merged_metadata)
        else:
            print(f"No metadata extracted from {filename}")
         
        time.sleep(0.5)

# Convert list of metadata rows into a single DataFrame
final_metadata_df = pd.DataFrame(all_metadata)


In [ ]:
final_metadata_df

In [ ]:
final_metadata_df.to_csv("metadata_resultsd.csv", index=False)